## Spam Filter For SMS Messages With Naive Bayes Algorithm

The goal of this project is to create a spam filter that can classify SMS messages as either SPAM or HAM(non-spam), using the multinomial Naive Bayes algorithm.

**Our target is to create a spam filter that can classify new messages with an accuracy greater than 80%**

For our program to classify the messages, we must first train it how humans classify messages as spam or ham. To do this we'll use a dataset of 5,572 SMS messages that are already classified by humans. You can download the dataset on [The UCI Machine Learning Repository here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) and learn more about it [here](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

Lets start by reading and exploring the dataset.

In [1]:
#Read the SMSSpamCollection Dataset into a Pandas DataFrame
import pandas as pd

df = pd.read_csv("SMSSpamCollection", header=None, sep='\t')

#set column names
df.columns = ['Label','SMS']

#print number of rows and columns
print("# Rows: {}\n# Columns: {}\n".format(df.shape[0], df.shape[1]))
#print first five rows
print(df.head())

# Rows: 5572
# Columns: 2

  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


## Ham/Spam Percentage

Let's find what percentage of the messages are spam and what percentage are ham.

In [2]:
#count ham and spam
num_ham = df[df['Label']== 'ham'].shape[0]
num_spam = df[df['Label']== 'spam'].shape[0]
print("ham count: {}".format(num_ham))
print("spam count: {}".format(num_spam))

#calculate ham and spam percentages.
ham_pct = num_ham / len(df) * 100
spam_pct = num_spam / len(df) * 100

print("ham percentage: {}%".format(round(ham_pct)))
print("spam percentage: {}%".format(round(spam_pct)))

ham count: 4825
spam count: 747
ham percentage: 87%
spam percentage: 13%


### Train-Test Split for Evaluating the Algorithm

To evaluate the performance of our spam filter we'll split the dataset into a train set and a test set.

- **Training set** to fit the model (our spam filter, which is Naive Bayes).


- **Testing set** to test the performance of the model. We'll compare predictions made on the test set with the true labels from the test set.

In [3]:
#Randomize the entire dataset.
rand_df = df.sample(frac=1, random_state=1)

#Split data 80% for training 20% for testing and reset index.
train_set = rand_df[0:4458].reset_index(drop=True)
test_set = rand_df[4458:].reset_index(drop=True)

#view the train and test datasets
print("num of train rows: {}\nnum of test rows: {}\n".format(train_set.shape[0], test_set.shape[0]))
print("train_set last five rows\n\n",train_set.tail())
print("==="*10)
print("\ntest_set first five rows\n\n", test_set.head())

num of train rows: 4458
num of test rows: 1114

train_set last five rows

      Label                                                SMS
4453   ham  Sorry, I'll call later in meeting any thing re...
4454   ham  Babe! I fucking love you too !! You know? Fuck...
4455  spam  U've been selected to stay in 1 of 250 top Bri...
4456   ham  Hello my boytoy ... Geeee I miss you already a...
4457   ham                           Wherre's my boytoy ? :-(

test_set first five rows

   Label                                                SMS
0   ham          Later i guess. I needa do mcat study too.
1   ham             But i haf enuff space got like 4 mb...
2  spam  Had your mobile 10 mths? Update to latest Oran...
3   ham  All sounds good. Fingers . Makes it difficult ...
4   ham  All done, all handed in. Don't know if mega sh...


In [4]:
#calculate and print the percentage of spam and ham in both the 
#training and the test set
train_spam_and_ham = train_set['Label'].value_counts(normalize=True)
print("train spam and ham pct: \n{}\n".format(round(train_spam_and_ham * 100)))

test_spam_and_ham = test_set['Label'].value_counts(normalize=True)
print("test spam and ham pct: \n{}".format(round(test_spam_and_ham * 100)))

train spam and ham pct: 
ham     87.0
spam    13.0
Name: Label, dtype: float64

test spam and ham pct: 
ham     87.0
spam    13.0
Name: Label, dtype: float64


The data was split proportionally so we can continue.

### Data Cleaning

Before we can train our model on the training set, we need to put it in a format that is easy to extract the information we need.

Instead of just two columns "Label" and "SMS", we'll add new columns, one for every unique word in all of the messages. The input values will be the number of words in the message for that row.

First a quick reminder of data format.

In [5]:
#first five of the train_set
train_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


We'll start by removing all the punctuation and making all letters in every word lowercase.

In [6]:
#Remove punctuation.
train_set['SMS'] = train_set['SMS'].str.replace('\W', ' ')
#Make all words lowercase
train_set['SMS'] = train_set['SMS'].str.lower()
train_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


Next we'll create a vocabulary for the messages in the training set. The vocabulary is the set of all unique words across all the mesasages in the training set.

In [7]:
#transform messages to a list of words that are strings.
train_set['SMS'] = train_set['SMS'].str.split()

#initialize a list to hold the vocabulary.
vocabulary = []

#loop over words in messages and add to vocabulary
for lis in train_set['SMS']:
    for w in lis:
        vocabulary.append(w)
        
#remove duplicates by transforming vocabulary to a set.
set_vocabulary = set(vocabulary)
#transform vocabulary back into a list.
vocabulary = list(set_vocabulary)
print(vocabulary[:20])
print(len(vocabulary))

['foley', '02', 'strongly', 'perpetual', 'fish', 'conserve', 'series', 'farm', 'embarassed', 'vale', 'wylie', 'drunk', 'swhrt', 'deviousbitch', 'clark', 'sat', 'chicken', 'desparate', 'checking', 'hi']
7783


Now that we have our vocabulary, we need to create a column for each unique word in our final DataFrame, filled with the number of words in each message.

To achieve this we'll first create a dictionary with keys as unique words from the vocabulary and the values will be a list the length of the training set, each index in the list will record the number of times the unique word appear in the messages with that same index. 

We'll then transform the dictionary into a DataFrame and finally 
concatenate it with our training dataset.

In [8]:
#create dict with unique words for keys and a list the len of the
#training set filled with zeros as the values.
word_counts_per_sms = {unique_word: [0]* len(train_set['SMS'])
                      for unique_word in vocabulary}

#count words in mesgs and update the word_count_per_sms dict.
for index, sms in enumerate(train_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        

#Transform word_counts_per_sms into a dataframe
word_counts = pd.DataFrame(word_counts_per_sms)

#concat to the training set
train_set = pd.concat([train_set, word_counts], axis=1)

train_set.head()

,Label,SMS,foley,02,strongly,perpetual,fish,conserve,series,farm,...,lapdancer,friendship,youdoing,losers,09058094454,recognises,okday,4ward,09061701461,nigeria
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Creating The Spam Filter

Now that we've cleaned the data and have a training set to work with, we can start building our spam filter. Below are the two probability equation we'll need to clasify our messages as spam or ham:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Here's a summary of the terms in the equations above:

\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}

P(Spam), P(Ham), NSpam, NHam and NVocabulary in the four equations will have the same values for every new message, so let's calculate them now.

In [9]:
#isolating spam messages in training set.
spam_sms = train_set[train_set['Label'] == 'spam']
#isolating ham messages in training set.
ham_sms = train_set[train_set['Label'] == 'ham']

#calculating NSpam.
n_spam = spam_sms['SMS'].apply(len).sum()
#calculating NHam.
n_ham = ham_sms['SMS'].apply(len).sum()        
#calculating NVocabulary
n_vocabulary = len(vocabulary)
#setting smoothing parameter to 1 (Laplace smoothing)
alfa = 1

#calculating probability of spam.
p_spam = len(spam_sms)/len(train_set)
#calculating probability of ham
p_ham = len(ham_sms)/len(train_set)

print("P(spam) = {}".format(round(p_spam, 2)))
print("P(ham) = {}".format(round(p_ham, 2)))

P(spam) = 0.13
P(ham) = 0.87


### \begin{equation}\text{Calculating }P(w_i|Spam)  \text{ and } P(w_i|Ham)\end{equation}
Above we calculated the terms in the equations that will have constant values for every new message (regardless of the message or each individual word in the message).

However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

Which means we can use our training set to calculate the probability for each word in our vocabulary.

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

These probability values are better known as the **parameters**, so lets now calculate the parameters using the following equations for P(wi|Spam) and P(wi|Ham):

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [10]:
#dicts to hold parameters
spam_parameter = {word: 0 for word in vocabulary}
ham_parameter = {word: 0 for word in vocabulary}

#calculating params and updating dicts.
for w in vocabulary:
    word_i_giv_spam = (spam_sms[w].sum() + alfa) / (n_spam.sum() + alfa * n_vocabulary)
    word_i_giv_ham = (ham_sms[w].sum() + alfa) / (n_ham.sum() + alfa * n_vocabulary)
    spam_parameter[w] = word_i_giv_spam
    ham_parameter[w] = word_i_giv_ham
    
#print first 10 spam parameters.    
first10_spam_params = {k: spam_parameter[k] for k in list(spam_parameter)[:10]}
print(first10_spam_params)

{'foley': 0.0001305880816610804, '02': 0.0003482348844295477, 'strongly': 4.3529360553693465e-05, 'perpetual': 4.3529360553693465e-05, 'fish': 4.3529360553693465e-05, 'conserve': 4.3529360553693465e-05, 'series': 4.3529360553693465e-05, 'farm': 4.3529360553693465e-05, 'embarassed': 4.3529360553693465e-05, 'vale': 4.3529360553693465e-05}


### Build The Spam Filter

We've calculated all the constants and parameters, so we can now build our spam filter using the first two equations we mentioned:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

In [11]:
import re

#function takes message as a string
def classify(string):
    message = re.sub('\W', ' ', string)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for w in message:
        if w in spam_parameter:
            p_spam_given_message *= spam_parameter[w]
        if w in ham_parameter:
            p_ham_given_message *= ham_parameter[w]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
                
#Use the classify() function to classify two new messages.
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [12]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Classifying The Test Set

So far our spam filter seem to be doing a good job, now let's determine how well it does clasifying the 1,114 messages in the test set. 

In [13]:
#change the function to return labels instead of pinting them.
def classify_test_set(string):
    message = re.sub('\W', ' ', string)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for w in message:
        if w in spam_parameter:
            p_spam_given_message *= spam_parameter[w]
        if w in ham_parameter:
            p_ham_given_message *= ham_parameter[w]
            
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return ' needs human classification'
        
#create new column in test set with predicted labels.    
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


### Measure Accuracy of the Spam Filter

Now that we have made our predictions, we can compare them with the true labels to see how accurate our spam filter is.

To make the measurement, we'll use **accuracy** as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [15]:
correct = 0

#total number of messages in the test set.
total = test_set.shape[0]

for index, row in test_set.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1
    
accruacy = correct/total
print(accruacy)

0.9874326750448833


Our initial goal was to achieve an accuracy greater than 80%, with our test set we managed to get **an accuracy of 98.74%**